In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import KFold
from tensorflow.keras import layers

tf.random.set_seed(1234)


In [3]:
DATASET_SIZE = 234
data_dir_ap = '/content/gdrive/MyDrive/compound_img2/AP/'
data_dir_L = '/content/gdrive/MyDrive/compound_img2/L/'


ap_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_ap,
  seed=42)


L_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_L,
  seed=42)

Found 234 files belonging to 2 classes.
Found 234 files belonging to 2 classes.


In [4]:
size = (150, 150)
ap_ds = ap_ds.map(lambda x, y: (tf.image.resize(x, size), y))
L_ds = L_ds.map(lambda x, y: (tf.image.resize(x, size), y))

In [5]:
x_ap = np.empty([1, 150, 150, 3])
y_ap = []
for x,y in ap_ds:
  x_ap = np.concatenate((x_ap, x), axis=0)
  y_ap = np.concatenate([y_ap, y], axis=0)

x_ap = x_ap[1:, :,:,:]

x_L = np.empty([1, 150, 150, 3])
y_L = []
for x,y in L_ds:
  x_L = np.concatenate((x_L, x), axis=0)
  y_L = np.concatenate([y_L, y], axis=0)

x_L = x_L[1:, :,:,:]

print(x_ap.shape)
print(x_L.shape)

(234, 150, 150, 3)
(234, 150, 150, 3)


In [62]:
#Ensure that the images are ordered the same way between the AP and L splits
print(y_L - y_ap)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
# Load weights pre-trained on ImageNet.
# Do not include the ImageNet classifier at the top.

base_model_ap = keras.applications.Xception(
    weights="imagenet",  
    input_shape=(150, 150, 3),
    include_top=False,
)  

base_model_L = keras.applications.Xception(
    weights="imagenet",  
    input_shape=(150, 150, 3),
    include_top=False,
) 

83689472/83683744 [==============================] - 0s 0us/step


In [56]:
data_augmentation_ap = keras.Sequential(
    [
        layers .experimental.preprocessing.RandomFlip("horizontal"), #L -> R flip
        layers.experimental.preprocessing.RandomRotation(0.05),
    ]
)

data_augmentation_L = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"), #L -> R flip
        layers.experimental.preprocessing.RandomRotation(0.05),
    ]
)

In [40]:
accscores = []
aucscores = []
raw_preds = []
gts = []

for kfold, (train, test) in enumerate(KFold(n_splits=5, shuffle=True).split(x_ap, y_ap)):
  tf.keras.backend.clear_session()

  base_model_ap.trainable = False
  base_model_ap._name="AP-View-Model"

  inputs_ap = keras.Input(shape=(150, 150, 3) ,name= "ap_input")
  
  augmented_ap = data_augmentation_ap(inputs_ap)  

  norm_layer_ap = keras.layers.experimental.preprocessing.Normalization()
  mean = np.array([127.5] * 3)
  var = mean ** 2
  augmented_ap = norm_layer_ap(augmented_ap)
  norm_layer_ap.set_weights([mean, var])

  augmented_ap = base_model_ap(augmented_ap, training=False)

  augmented_ap = keras.layers.GlobalAveragePooling2D()(augmented_ap)
  augmented_ap = keras.layers.Dropout(0.2)(augmented_ap)  # Regularize with dropout
  output_ap = keras.layers.Dense(1, activation='sigmoid')(augmented_ap)

  model_ap = keras.Model(inputs_ap, output_ap)
  #-----------Model is now defined, moving on to training---------

  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

  model_ap.compile(
      optimizer=keras.optimizers.Adam(),
      loss=keras.losses.BinaryCrossentropy(from_logits=True),
      metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.AUC()],
  )

  epochs = 200

  #First fit with frozen weights
  model_ap.fit(x_ap[train], y=y_ap[train], epochs=epochs, callbacks=[callback], validation_split=0.2)

  base_model_ap.trainable = True

  model_ap.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.AUC()],
  ) 

  epochs = 40

  #Second fit
  model_ap.fit(x_ap[train], y=y_ap[train], epochs=epochs, callbacks=[callback], validation_split=0.2)

  #Evaluate
  predictions = model_ap.predict(x_ap[test])
  raw_preds.append(predictions)
  gts.append(y_ap[test])
  scores = model_ap.evaluate(x_ap[test], y=y_ap[test])


  print("%s: %.2f%%" % (model_ap.metrics_names[0], scores[0]*100))
  accscores.append(scores[1] * 100)
  aucscores.append(scores[2])
 
print("%.2f%% (+/- %.2f%%)" % (np.mean(accscores), np.std(accscores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(aucscores), np.std(aucscores)))

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:5017: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


5/5 [==============================] - 5s 313ms/step - loss: 1.4516 - binary_accuracy: 0.5638 - auc: 0.5207 - val_loss: 0.4321 - val_binary_accuracy: 0.8158 - val_auc: 0.8936
Epoch 2/100
5/5 [==============================] - 0s 76ms/step - loss: 0.2616 - binary_accuracy: 0.9060 - auc: 0.9753 - val_loss: 0.4579 - val_binary_accuracy: 0.8947 - val_auc: 0.9188
Epoch 3/100
5/5 [==============================] - 0s 76ms/step - loss: 0.0619 - binary_accuracy: 0.9799 - auc: 0.9986 - val_loss: 0.6518 - val_binary_accuracy: 0.8947 - val_auc: 0.9272
Epoch 4/100
5/5 [==============================] - 0s 77ms/step - loss: 0.0129 - binary_accuracy: 0.9933 - auc: 1.0000 - val_loss: 0.7668 - val_binary_accuracy: 0.8947 - val_auc: 0.9300
Epoch 5/100
5/5 [==============================] - 0s 77ms/step - loss: 0.0176 - binary_accuracy: 0.9933 - auc: 1.0000 - val_loss: 0.8017 - val_binary_accuracy: 0.8947 - val_auc: 0.9300
Epoch 6/100
5/5 [==============================] - 0s 75ms/step - loss: 0.0148 - 

In [54]:
def flatten(t):
    return [item for sublist in t for item in sublist]

raw_preds_ap = flatten(flatten(raw_preds))
gts_ap = flatten(gts)


In [59]:
accscores = []
aucscores = []
raw_preds = []
gts = []

for kfold, (train, test) in enumerate(KFold(n_splits=5, shuffle=True).split(x_L, y_L)):
  tf.keras.backend.clear_session()
  
  base_model_L.trainable = False
  base_model_L._name="L-Model"

  inputs_L = keras.Input(shape=(150, 150, 3), name="L_input")
  augmented_L = data_augmentation_L(inputs_L)  

  norm_layer_L = keras.layers.experimental.preprocessing.Normalization()
  mean = np.array([127.5] * 3)
  var = mean ** 2
  augmented_L = norm_layer_L(augmented_L)
  norm_layer_L.set_weights([mean, var])
  augmented_L = base_model_L(augmented_L, training=False)


  augmented_L = keras.layers.GlobalAveragePooling2D()(augmented_L)
  augmented_L = keras.layers.Dropout(0.2)(augmented_L)  # Regularize with dropout
  output_L = keras.layers.Dense(1, activation='sigmoid')(augmented_L)
  model_L = keras.Model(inputs_L, output_L)

  #-----------Model is now defined, moving on to training---------

  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

  model_L.compile(
      optimizer=keras.optimizers.Adam(),
      loss=keras.losses.BinaryCrossentropy(from_logits=True),
      metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.AUC()],
  )

  epochs = 200

  #First fit with frozen weights
  model_L.fit(x_L[train], y=y_L[train], epochs=epochs, callbacks=[callback], validation_split=0.2)

  base_model_L.trainable = True

  model_L.compile(
      optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
      loss=keras.losses.BinaryCrossentropy(from_logits=True),
      metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.AUC()],
  )

  epochs = 40

  #Second fit
  model_L.fit(x_L[train], y=y_L[train], epochs=epochs, callbacks=[callback], validation_split=0.2)

  #Evaluate
  predictions = model_L.predict(x_L[test])
  raw_preds.append(predictions)
  gts.append(y_L[test])
  scores = model_L.evaluate(x_L[test], y=y_L[test])


  print("%s: %.2f%%" % (model_L.metrics_names[0], scores[0]*100))
  accscores.append(scores[1] * 100)
  aucscores.append(scores[2])
 
print("%.2f%% (+/- %.2f%%)" % (np.mean(accscores), np.std(accscores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(aucscores), np.std(aucscores)))

Epoch 1/200


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:5017: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


5/5 [==============================] - 4s 289ms/step - loss: 0.6763 - binary_accuracy: 0.5772 - auc_10: 0.5931 - val_loss: 0.6902 - val_binary_accuracy: 0.6842 - val_auc_10: 0.6750
Epoch 2/200
5/5 [==============================] - 0s 78ms/step - loss: 0.5970 - binary_accuracy: 0.6510 - auc_10: 0.7517 - val_loss: 0.7155 - val_binary_accuracy: 0.6842 - val_auc_10: 0.6847
Epoch 3/200
5/5 [==============================] - 0s 78ms/step - loss: 0.5628 - binary_accuracy: 0.6980 - auc_10: 0.7801 - val_loss: 0.6553 - val_binary_accuracy: 0.6579 - val_auc_10: 0.7153
Epoch 4/200
5/5 [==============================] - 0s 77ms/step - loss: 0.5181 - binary_accuracy: 0.7987 - auc_10: 0.8406 - val_loss: 0.6431 - val_binary_accuracy: 0.7105 - val_auc_10: 0.7375
Epoch 5/200
5/5 [==============================] - 0s 77ms/step - loss: 0.4950 - binary_accuracy: 0.7852 - auc_10: 0.8527 - val_loss: 0.6409 - val_binary_accuracy: 0.6842 - val_auc_10: 0.7375
Epoch 6/200
5/5 [==============================] - 

In [60]:
raw_preds_L = flatten(flatten(raw_preds))
gts_L = flatten(gts)

In [64]:
accscores = []
aucscores = []
raw_preds = []
gts = []

for kfold, (train, test) in enumerate(KFold(n_splits=5, shuffle=True).split(x_L, y_L)):

  tf.keras.backend.clear_session()

  base_model_ap.trainable = False
  base_model_ap._name="AP-View-Model"

  inputs_ap = keras.Input(shape=(150, 150, 3) ,name= "ap_input")
  
  augmented_ap = data_augmentation_ap(inputs_ap)  

  norm_layer_ap = keras.layers.experimental.preprocessing.Normalization()
  mean = np.array([127.5] * 3)
  var = mean ** 2
  augmented_ap = norm_layer_ap(augmented_ap)
  norm_layer_ap.set_weights([mean, var])

  augmented_ap = base_model_ap(augmented_ap, training=False)

  augmented_ap = keras.layers.GlobalAveragePooling2D()(augmented_ap)
  augmented_ap = keras.layers.Dropout(0.2)(augmented_ap)  # Regularize with dropout


  base_model_L.trainable = False
  base_model_L._name="L-Model"

  inputs_L = keras.Input(shape=(150, 150, 3), name="L_input")
  augmented_L = data_augmentation_L(inputs_L)  

  norm_layer_L = keras.layers.experimental.preprocessing.Normalization()
  mean = np.array([127.5] * 3)
  var = mean ** 2
  augmented_L = norm_layer_L(augmented_L)
  norm_layer_L.set_weights([mean, var])
  augmented_L = base_model_L(augmented_L, training=False)


  augmented_L = keras.layers.GlobalAveragePooling2D()(augmented_L)
  augmented_L = keras.layers.Dropout(0.2)(augmented_L)  # Regularize with dropout

  cat = layers.concatenate([augmented_L, augmented_ap])
  cat = layers.Flatten()(cat)
  dense = layers.Dense(512)(cat)
  dense = layers.LeakyReLU(alpha=0.1)(dense)
  dense = layers.Dropout(0.5)(dense)
  outputs = layers.Dense(1, activation='sigmoid')(dense)

  model = keras.Model(inputs=[inputs_ap, inputs_L], outputs=outputs,)

  #-----------Model is now defined, moving on to training---------

  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

  model.compile(
      optimizer=keras.optimizers.Adam(),
      loss=keras.losses.BinaryCrossentropy(from_logits=True),
      metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.AUC()],
  )

  epochs = 200

  #First fit with frozen weights
  model.fit(x=[x_ap[train], x_L[train]],y=y_ap[train] , callbacks=[callback], epochs=epochs, validation_split=0.2)

  base_model_L.trainable = True
  base_model_ap.trainable = True

  model.compile(
      optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
      loss=keras.losses.BinaryCrossentropy(from_logits=True),
      metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.AUC()],
  )

  epochs = 40

  #Second fit
  model.fit(x=[x_ap[train], x_L[train]],y=y_ap[train] , callbacks=[callback], epochs=epochs, validation_split=0.2)

  #Evaluate
  predictions = model.predict([x_ap[test], x_L[test]])
  raw_preds.append(predictions)
  gts.append(y_L[test])
  scores = model.evaluate(x=[x_ap[test], x_L[test]], y=y_L[test])
  

  print("%s: %.2f%%" % (model_L.metrics_names[0], scores[0]*100))
  accscores.append(scores[1] * 100)
  aucscores.append(scores[2])
 
print("%.2f%% (+/- %.2f%%)" % (np.mean(accscores), np.std(accscores)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(aucscores), np.std(aucscores)))

Epoch 1/200


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:5017: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


5/5 [==============================] - 7s 533ms/step - loss: 0.7651 - binary_accuracy: 0.8054 - auc: 0.9010 - val_loss: 3.6515 - val_binary_accuracy: 0.8947 - val_auc: 0.9217
Epoch 2/200
5/5 [==============================] - 1s 149ms/step - loss: 0.0224 - binary_accuracy: 0.9933 - auc: 1.0000 - val_loss: 4.6877 - val_binary_accuracy: 0.8947 - val_auc: 0.9188
Epoch 3/200
5/5 [==============================] - 1s 147ms/step - loss: 0.0180 - binary_accuracy: 0.9933 - auc: 1.0000 - val_loss: 6.6989 - val_binary_accuracy: 0.8684 - val_auc: 0.8986
Epoch 4/200
5/5 [==============================] - 1s 149ms/step - loss: 3.6132e-11 - binary_accuracy: 1.0000 - auc: 1.0000 - val_loss: 8.7679 - val_binary_accuracy: 0.8421 - val_auc: 0.8580
Epoch 5/200
5/5 [==============================] - 1s 190ms/step - loss: 3.1013e-12 - binary_accuracy: 1.0000 - auc: 1.0000 - val_loss: 10.0617 - val_binary_accuracy: 0.8421 - val_auc: 0.8580
Epoch 6/200
5/5 [==============================] - 1s 151ms/step - l

In [65]:
raw_preds_combined = flatten(flatten(raw_preds))
gts_combined = flatten(gts)